# 📘 Notebook 04: Data Preprocessing

## 🎯 What I Plan to Achieve in This Notebook

In this notebook, I will prepare my raw Spotify dataset for machine learning by performing the following preprocessing steps:

1. **Handle Missing Values** – Identify and treat any null/NaN values in the dataset
2. **Handle Duplicate Records** – Remove or manage duplicate entries to ensure data integrity
3. **Outlier Detection & Treatment** – Address extreme values identified during EDA (especially in loudness, instrumentalness, speechiness)
4. **Data Type Validation** – Ensure all features have appropriate data types for modeling
5. **Feature Engineering (Basic)** – Create any derived features if needed based on EDA insights
6. **Data Quality Checks** – Verify data consistency and integrity before moving to encoding

---

## Why Preprocessing is Necessary

### 1️⃣ **Machine Learning Models Cannot Handle "Dirty" Data**

- Missing values break algorithms – Most ML models (Random Forest, Linear Regression, etc.) cannot process NaN or null values
- Duplicates cause data leakage – Duplicate records can artificially inflate model performance during training and evaluation
- Outliers distort patterns – Extreme values can skew the model's understanding of normal data patterns

### 2️⃣ **Based on My EDA Findings**

From my exploratory analysis, I discovered several data quality issues:

- **Outliers in multiple features:**
  - `loudness` has extreme negative values below -40 dB
  - `instrumentalness` shows many values concentrated at 1.0 (fully instrumental)
  - `speechiness` has outliers above 0.8 (spoken word content)
  - `liveness` shows unusual high values close to 1.0

- **Weak feature correlations with target:**
  - Highest correlation is only **0.1275** (instrumentalness)
  - This suggests I may need feature engineering or interaction terms

- **Bimodal distributions:**
  - `acousticness` shows two distinct peaks (acoustic vs electric instruments)
  - `energy` shows bimodal distribution (calm vs energetic tracks)

### 3️⃣ **Ensures Model Reliability**

- Improved generalization – Clean data helps the model learn true patterns, not noise
- Faster training – Removing outliers and duplicates reduces computational overhead
- Better interpretability – Clean data makes it easier to understand model decisions

---

## 📋 Preprocessing Strategy

Based on my EDA insights, I will adopt the following approach:

### **Missing Values:**
- First, check if any features have missing data
- If found, decide between imputation(mean/median/mode) or deletion based on percentage of missing values

### **Duplicates:**
- Identify duplicate rows based on all features or specific key columns
- Remove exact duplicates while preserving unique records

### **Outliers:**
- Use IQR (Interquartile Range) method to identify outliers
- Use domain knowledge from EDA (e.g., loudness below -50 dB is unrealistic)
- Either cap/floor extreme values or remove them (will decide based on impact)

### **Data Type Validation:**
- Ensure numerical features are `float64` or `int64`
- Ensure categorical features (like `explicit`, `key`, `mode`) are properly typed
- Convert `time_signature` if needed

---

## Important Note

This notebook focuses **ONLY** on data cleaning and quality assurance. The following tasks will be handled in separate notebooks:

- **Encoding categorical variables** → Notebook 05
- **Feature scaling/normalization** → Notebook 06  
- **Feature selection** → Notebook 07
- **Train-test split** → Before modeling phase


---

## 📊 Expected Outcome

By the end of this notebook, I will have:

✅ A clean dataset with **no missing values**  
✅ **No duplicate records**  
✅ **Outliers handled** appropriately  
✅ **Validated data types** for all features  
✅ A dataset ready for encoding and scaling in subsequent notebooks  

This clean dataset will serve as the foundation for building robust machine learning models in the modeling phase.

<p align="center">
  <img src="../assets/dividerlines.png" width="600"/>
</p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('../data/spotify_dedup.csv')

### Handle Missing Values
- Based on the dataset size ($n=89,740$), if missing values are less than 1%, I will drop them. Otherwise, apply median imputation for numerical features.

In [2]:
# Check for missing values
null_counts = df.isnull().sum()
print("Missing values per column:\n", null_counts[null_counts > 0])

# Strategy: Drop rows with missing values (usually minimal in this dataset)
df = df.dropna()
print(f"Shape after handling missing values: {df.shape}")

Missing values per column:
 Series([], dtype: int64)
Shape after handling missing values: (89740, 23)


In [14]:
missing = df.isnull().sum().sum()
if missing == 0:
    print("✅ CHECK: No missing values")
else:
    print(f"❌ CHECK FAILED: {missing} missing values found")

✅ CHECK: No missing values


### Checking for Duplicates
I already removed the duplicates in 02_EDA_Statistical_Analysis.ipynb


In [3]:
# Detailed duplicate check
exact_duplicates = df.duplicated().sum()
id_duplicates = df.duplicated(subset=['track_id']).sum()

print(f"Exact duplicates: {exact_duplicates}")
print(f"Duplicate Track IDs: {id_duplicates}")

Exact duplicates: 0
Duplicate Track IDs: 0


### Outlier Detection & Treatment
From EDA,  loudness, instrumentalness, and speechiness have extreme distributions. We will use a "Capping" approach to handle outliers without losing the data points entirely.

#### What is Capping?
*Capping (or Winsorization) means limiting extreme values to a defined boundary instead of removing them.*

**Context:** You do *not* delete outlier rows; you replace extreme values with a reasonable maximum or minimum.

*Example:*
If your **loudness** lower bound is -25 and a song has loudness = -40, after capping:  
`-40 → -25`

In [6]:
def handle_outliers_iqr(df, column):
    # 1. Compute first and third quartiles
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    # 2. Interquartile Range (IQR)
    IQR = Q3 - Q1
    # 3. Define lower and upper bounds using 1.5 * IQR rule
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Capping values at the bounds
    #    Values below lower_bound → set to lower_bound
    #    Values above upper_bound → set to upper_bound
    df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
    df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])
    return df

# Apply to specific features identified in EDA
outlier_cols = ['loudness', 'speechiness', 'liveness']
for col in outlier_cols:
    df = handle_outliers_iqr(df, col)

print("Outlier treatment complete.")

Outlier treatment complete.


Outliers were treated using an IQR-based capping approach. Instead of removing extreme observations, values beyond 1.5×IQR were capped at the lower and upper bounds. This approach reduces the influence of extreme values while preserving the full dataset.

### Data Type Validation
Ensuring that booleans are integers and IDs are strings.

In [7]:
# Convert explicit from bool to int (0/1)
df['explicit'] = df['explicit'].astype(int)

# Ensure track_genre and names are strings
categorical_cols = ['track_id', 'artists', 'album_name', 'track_name', 'track_genre']
for col in categorical_cols:
    df[col] = df[col].astype(str)

print(df.dtypes)

Unnamed: 0             int64
track_id              object
artists               object
album_name            object
track_name            object
popularity             int64
duration_ms            int64
explicit               int64
danceability         float64
energy               float64
key                    int64
loudness             float64
mode                   int64
speechiness          float64
acousticness         float64
instrumentalness     float64
liveness             float64
valence              float64
tempo                float64
time_signature         int64
track_genre           object
duration_min         float64
duration_category     object
dtype: object


#### Feature Engineering (Basic)
Since individual correlations with popularity are weak ($r < 0.15$), we will create interaction terms to help the model capture complex patterns.

This code performs feature engineering by creating interaction variables, binary categories, and log transforms to expose hidden patterns and normalize distributions within your music dataset. These transformations translate raw data into more descriptive signals, allowing machine learning models to learn faster and make more accurate predictions.

In [8]:
# A. Interaction Features
df['energy_x_danceability'] = df['energy'] * df['danceability']
df['loudness_x_energy'] = df['loudness'] * df['energy']
df['valence_x_danceability'] = df['valence'] * df['danceability']

# B. Binary Indicators (Based on EDA bimodal distributions)
df['is_instrumental'] = (df['instrumentalness'] > 0.5).astype(int)
df['has_vocals'] = (df['instrumentalness'] < 0.2).astype(int)
df['is_speech_heavy'] = (df['speechiness'] > 0.33).astype(int)

# C. Log Transforms for Skewed Data (if needed)
# Adding a small constant to avoid log(0)
df['tempo_log'] = np.log1p(df['tempo'])

print(f"New features created. Total columns: {len(df.columns)}")

New features created. Total columns: 30


In [9]:
new_corr = df[['energy_x_danceability', 'loudness_x_energy', 'is_instrumental', 'popularity']].corr()['popularity']
print("Correlation of new features with Popularity:\n", new_corr)

Correlation of new features with Popularity:
 energy_x_danceability    0.038122
loudness_x_energy        0.057023
is_instrumental         -0.120739
popularity               1.000000
Name: popularity, dtype: float64


In [11]:
# Save the engineered dataset as a new checkpoint
df = df.reset_index(drop=True)
df.to_csv('../data/spotify_preprocessed.csv', index=False)
print(f"Total columns saved: {len(df.columns)}")

Total columns saved: 30


In [12]:
df.head(5)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,track_genre,duration_min,duration_category,energy_x_danceability,loudness_x_energy,valence_x_danceability,is_instrumental,has_vocals,is_speech_heavy,tempo_log
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,0,0.676,0.4610,...,acoustic,3.844433,3–4 min,0.311636,-3.109906,0.483340,0,1,0,4.487703
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,0,0.420,0.1660,...,acoustic,2.493500,2–3 min,0.069720,-2.861010,0.112140,0,1,0,4.362958
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,0,0.438,0.3590,...,acoustic,3.513767,3–4 min,0.157242,-3.494506,0.052560,0,1,0,4.348108
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,0,0.266,0.0596,...,acoustic,3.365550,3–4 min,0.015854,-1.081360,0.038038,0,1,0,5.208064
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,0,0.618,0.4430,...,acoustic,3.314217,3–4 min,0.273774,-4.288683,0.103206,0,1,0,4.795369
